In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import os
os.environ['R_HOME'] = 'D:/Program Files/R-4.5.0' #@TODO include in some setup file

from simulation_engine.scenarios.iv.binary_iv import BinaryIV
from simulation_engine.scenarios.iv.continuous_iv import ContinuousIV
from simulation_engine.scenarios.iv.base_iv import IVScenario
from simulation_engine.util.plotting_util import PlottingUtil
from simulation_engine.algorithms.causaloptim import Causaloptim
import pandas as pd
pd.set_option('display.max_columns', None)

#disable warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from simulation_engine.algorithms.zaffalonbounds import ZaffalonBounds

In [8]:
data = BinaryIV.generate_data_rolling_ate(N_simulations=4)

In [ ]:
binaryIV = BinaryIV('IV Dag', data)

runtimes = binaryIV.run_all_bounding_algorithms(['ATE_zaffalonbounds'])

Running ATE_zaffalonbounds...
ATE_zaffalonbounds completed in 27.13 seconds.
Total runtime: 27.13 seconds.


,seed,intercept_X,intercept_Y,b_Z,b_U_X,b_X_Y,b_U_Y,ATE_true,PNS_true,p_Y1_mean,p_Y0_mean,p_U,p_Z,Z,U,X,Y,entropy_Z,entropy_U,entropy_X,entropy_Y,ATE_zaffalonbounds_bound_lower,ATE_zaffalonbounds_bound_upper,ATE_zaffalonbounds_bound_valid,ATE_zaffalonbounds_bound_width,ATE_zaffalonbounds_bound_failed
0,392922,0,0,-0.982364,0.426819,-5.0,0.620694,-0.552051,0.003745,0.009001,0.561053,0.406843,0.065883,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, ...","[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, ...",0.395093,0.974352,0.999954,0.878832,-0.5980,-0.0007,True,0.5973,False
1,471430,0,0,-0.477779,1.008529,-2.5,-0.021602,-0.420734,0.037624,0.074547,0.495280,0.878992,0.829342,"[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...",0.639073,0.546365,0.963497,0.784919,-0.5333,-0.0042,True,0.5291,False
2,618742,0,0,1.274890,0.593062,0.0,0.998910,0.000000,0.214723,0.652819,0.652819,0.669764,0.420808,"[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...","[1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, ...",0.952195,0.922892,0.914926,0.953694,0.0603,0.3209,False,0.2606,False
3,608132,0,0,-0.268058,1.005507,2.5,-0.226724,0.462378,0.503726,0.906954,0.444576,0.985463,0.004430,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, ...","[1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...",0.080793,0.130059,0.888502,0.756499,0.0084,0.6067,True,0.5983,False


In [7]:
algorithms = list(runtimes['runtimes'].keys())
PlottingUtil.print_bound_statistics(binaryIV.data, algorithms)
PlottingUtil.plot_smoothed_query_vs_bounds(binaryIV.data, 'ATE', algorithms, 1)

Algorithm: ATE_zaffalonbounds
  Fail Rate: 0.00%
  Invalid Rate: 0.00%
  Net Bound Width: 0.32200000000000006
Error: The 'b_X_Y' column has constant or invalid values. Cannot plot.


In [9]:
invalids = binaryIV.data[binaryIV.data['causaloptim_bound_valid'] == False]
invalIV = BinaryIV('IV Dag', invalids)
invalIV.bound_ate_causaloptim()
invalIV.data

caputured: R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In boundsfunc(p00_0 = p00_0, p00_1 = p00_1, p10_0 = p10_0, p10_1 = p10_1,  :
R[write to console]: 
 
R[write to console]:  Invalid bounds! Data probably does not satisfy the assumptions in the DAG!


Invalid Bounds detected
caputured: 
caputured: 


,seed,intercept_X,intercept_Y,b_Z,b_U_X,b_X_Y,b_U_Y,ATE_true,p_Y1_mean,p_Y0_mean,p_U,p_Z,Z,U,X,Y,entropy_Z,entropy_U,entropy_X,entropy_Y,causaloptim_bound_lower,causaloptim_bound_upper,causaloptim_bound_valid,causaloptim_bound_width,causaloptim_bound_failed
3,944243,0,0,2.264300,-0.893513,-4.7,-0.207546,-0.448266,0.007581,0.455847,0.866774,0.819945,"[1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",0.705637,0.599739,0.849943,0.517753,-0.344369,-0.348804,False,-0.004435,False
49,877939,0,0,-1.917451,1.823300,-0.1,-1.612648,-0.013956,0.168286,0.182243,0.949875,0.809191,"[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.697269,0.277840,0.998049,0.614823,-0.157216,-0.107379,False,0.049838,False
90,220567,0,0,-1.028126,-2.033834,4.0,0.054696,0.472237,0.982737,0.510499,0.792267,0.222949,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","[1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",0.737642,0.781481,0.653109,0.963497,-0.282440,0.451923,False,0.734363,False
